# HPE Container Platform API series  - Lab 3 (Spark)
## Launching cloud-native stateless applications and non-cloud-native stateful applications, programmatically through REST API calls, as a Kubernetes Tenant user.


**Requirements:**
- HPE Container Platform deployment
- IP address of FQDN of the HPE Container Platform's controller host
- a Kubernetes tenant user account   

**Utilities:**   
- cURL  
- Jupyter Notebook server with bash kernel installed
- kubectl, kubectl hpecp plug-in

**Definitions:**
- *HPE Container Platform* is an enterprise-grade container platform designed to deploy both cloud-native and non-cloud-native applications whether on-premises, at the edge, in multiple public clouds, or in a hybrid model. This makes the HPE Container Platform ideal for helping application developers and data scientists accelerate their application development and deployment on **containers**, on-demand through a self-service portal and a RESTful API that surfaces programmable access. To learn more about HPE Container Platform visit the [HPE DEV portal](https://developer.hpe.com/platform/hpe-container-platform/home) and check out the blog articles.

- *tenant:* A tenant is a group of users created by the platform administrator. A tenant can represent for example, an office location, a business unit, an organization, a project, an application. A tenant is allocated a set of resources (CPU/GPU, RAM, Storage, App Store images, Kubernetes cluster) by the platform administrator. All the resources used by a tenant are not shared with other tenants. A tenant user is granted the role of member or admin for the tenant.


HPE Container Platform provides two types of tenants:  
* Big data, AI/ML tenants that exist within the context of HPE Container Platform. This type of tenants are also known as **EPIC** (Elastic Private Instant Cluster) tenants.
* Kubernetes tenants that exist within the context of one or more Kubernetes clusters managed by HPE Container Platform.
  
Here, in this lab part, I will cover how **Kubernetes tenant** users can deploy, using REST API, both cloud-native stateless, microservices applications, and non-cloud-native distributed stateful AI/analytics **KubeDirector** applications on **Kubernetes** cluster managed by HPE Container Platform. Tenant users will then use kubectl to interact directly with the Kubernetes cluster in the context of their tenant.

KubeDirector, also known as Kubernetes Director, is a key component of HPE Container Platform. KubeDirector is an open source project initiated by HPE (BlueData) that enables the running of non-cloud-native stateful analytics workloads on Kubernetes. 

## The HPE Container Platform API Reference
The HPE Container Platform REST API allows you to achieve multiple actions programmatically, from performing administrative tasks, deploying cloud-native and non-cloud-native applications to scoring trained Machine Learning models.

Before you can call the HPE Container Platform API, you need to know what calls can be placed. The REST API reference documentation describes each object type, along with the supported operations, request input parameters, response model and response codes. 
To access the REST API reference documentation, obtain the IP address or hostname of the current active HPE Container Platform controller host from the administrator of the platform. Then in a web browser, navigate to the following URL:

``` 
http(s)://[Controller-IP-address-name]/apidocs
```

All the REST API calls are in the form: 
``` 
An HTTP VERB such as (GET , POST, DELETE, PUT, PATCH, UPDATE),  
A target API object URL: http(s)://[Controller-IP-address]:8080/api/v2/[object]
```

## Session Authentication in a multi-tenant environment
With the exception of some API calls, most of the REST API calls you can do against the HPE Container Platform API requires authentication. HPE Container Platform uses a *‘session location’* to use as operation context. In a multi-tenant environment, you request an authentication session location by issuing an authentication request in the following form:
* Call the API to request a new login session, providing username/password credentials as well as the Tenant name in the JSON body.  The user must be a valid tenant user credentials with a role (member or admin) in the requested tenant. 
* Extract the resource path of the created *session location* object from the JSON response header,
* For each subsequent call, set a new HTTP Header with its key set to *X-BDS-SESSION* and its value set to the session location value and used as the *working tenant* context. 

``` 
Note: the session location will expire after 24 hours. 
```   

If you are not already familiar with REST API calls, I encourage you to check-out the [Understanding API basics and the value they provide](https://developer.hpe.com/blog/understanding-api-basics-and-the-value-they-provide) blog on HPE Developer Community portal. It explains you REST API concepts such as HTTP verbs you call against the API, the headers, and payloads used when making API calls.  

**cURL:** You will use cURL to make API requests. Information on cURL can be found [here](https://curl.haxx.se/)

#### Initialize the environment.

**IMPORTANT: Before running the next cells, please make sure to adjust the environment variables below according to your Student ID, tenant username and password.**

In [1]:
#
# environment variables to be adjusted by the student
#
student="denis" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="demouser" # your tenant login credentials - username and password (it matches your Notebook Student account)
password="password"
#
# fixed environment variables setup by the HPE CP administrator
#
controller_endpoint="controller.hpedevlab.net:8080"
controller_host="controller.hpedevlab.net"
tenantname="K8sHackTenant"
k8sClusterId="1"  #this is the K8s Cluster Id provided by the HPE CP admisnistrator and assigned to your K8s tenant.
helloWorldApp="hello-world-app.yaml" # the application manifest you will deploy in this lab
tensorFlowApp="tensorflow-notebook-config-cluster.yaml" # the kubedirector application cluster configuration
sparkApp="spark221e2-stor-config-cluster.yaml" # the kubedirector application cluster configuration Spark221e2

#### Authenticate as Tenant user in the specified tenant:

In [2]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

This is your session location:  /api/v2/session/242bf46b-926c-426e-88d3-45c42fb51a20
This is your session_Id: 242bf46b-926c-426e-88d3-45c42fb51a20


#### Make a quick check to ensure you can make REST API calls within your tenant working context:
Here you will fetch information about your session you have just established.

In [3]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq  #using jq to pretty print the JSON reponse of the API call 

{
  "_links": {
    "self": {
      "href": "/api/v2/session/242bf46b-926c-426e-88d3-45c42fb51a20"
    },
    "all_sessions": {
      "href": "/api/v2/session"
    }
  },
  "user": "/api/v1/user/60",
  "user_name": "demouser",
  "tenant": "/api/v1/tenant/5",
  "tenant_name": "K8sHackTenant",
  "role": "/api/v1/role/3",
  "role_name": "Member",
  "expiry": "2020-4-17 00:19:58",
  "expiry_time": 1587075598,
  "is_site_admin_view": false,
  "is_cluster_superuser": false
}


## Deploying a simple cloud-native stateless application

#### -1- Deploy Hello World containerized application
You will deploy a simple Hello World application on the Kubernetes cluster made available to your tenant. The REST API (POST) call requires the **kubectl** operation type (create or apply) and the application manifest (YAML file that describes the attributes of the application) in a base64 encoded form.  

As you are all sharing the same tenant context and Kubernetes cluster resources, let's make sure your application deployment name will be unique among the tenant users. Here we replace the string "example" with your "username".

In [4]:
sed -i "s/example/${username}/g" $helloWorldApp
cat $helloWorldApp

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world-demouser
spec:
  selector:
    matchLabels:
      run: load-balancer-demouser
  replicas: 2
  template:
    metadata:
      labels:
        run: load-balancer-demouser
    spec:
      containers:
        - name: hello-world-demouser
          image: gcr.io/google-samples/node-hello:1.0
          ports:
            - containerPort: 8080
              protocol: TCP
---
apiVersion: v1
kind: Service
metadata:
  name: hello-world-service-demouser
spec:
  selector:
    run: load-balancer-demouser
  ports:
  - name: http-hello
    protocol: TCP
    port: 8080
    targetPort: 8080
  type: NodePort


In [5]:
#encode the application description file in base64
myapp=$(base64 $helloWorldApp)
#echo $myapp

In [6]:
curl -k -s --request POST "https://${controller_endpoint}/api/v2/k8scluster/${k8sClusterId}/kubectl" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"data": "'"$myapp"'",
"op": "apply"
}'

deployment.apps/hello-world-demouser created
service/hello-world-service-demouser created

After a minute or so, you should get the message *Deployment and service created* on the Kubernetes cluster.   

You will now directly interact with the Kubernetes cluster using **kubectl** (the command line configuration tool for Kubernetes).  

In the next steps, you will use kubectl commands in the context of your tenant user account and get the service endpoints of your application to connect to it.

#### -2- Establish the kubectl authenticated context using kubectl hpecp plugin

The kubectl hpecp plug-in is an extension of the standard kubectl command line. The plugin handles HPECP login and session token management. It is used to establish **HPECP-authenticated** kubectl requests to HPECP-managed Kubernetes services.   

Let's get the kubeconfig file for your working tenant context. The kubeconfig file contains kubernetes context that tenant users can interact with, based on their assigned role (tenant admin or tenant member).

In [7]:
#kubectl version --client
#kubectl hpecp version
kubectl hpecp refresh --insecure-skip-tls-verify ${controller_host} --hpecp-user="${username}" --hpecp-pass="${password}"



Retrieved new Kube Config from HPECP server at controller.hpedevlab.net:8080.
The KUBECONFIG environment variable HAS NOT been set.
Your current session WILL NOT have the new configuration.
To persist these changes by loading all current Kube Config
values into your default Kube Config file, run the
following command:

    KUBECONFIG="/home/denis/.kube/.hpecp/controller.hpedevlab.net/config:/home/denis/.kube/config-backup" kubectl config view --raw > /home/denis/.kube/config

To persist these changes by changing your local KUBECONFIG
environment variable, run the following command:

    export KUBECONFIG="/home/denis/.kube/.hpecp/controller.hpedevlab.net/config"

CAUTION - both of these commands will OVERWRITE your current
Kube Config settings. This is probably what you want, but
to confirm that this command will not break your system,
run the following command to view the resulting Kube
Config file:

    KUBECONFIG="/home/denis/.kube/.hpecp/controller.hpedevlab.net/config:/home/deni

In [8]:
#define the Kubeconfig file as a shell environment variable
export KUBECONFIG="/home/${student}/.kube/.hpecp/${controller_host}/config"
echo $KUBECONFIG

/home/denis/.kube/.hpecp/controller.hpedevlab.net/config


Check your working tenant context

In [9]:
kubectl config view

apiVersion: v1
clusters:
- cluster:
    insecure-skip-tls-verify: true
    server: https://gateway1.etc.fr.comm.hpecorp.net:9500
  name: k8scluster1
contexts:
- context:
    cluster: k8scluster1
    namespace: k8shacktenant
    user: HPECP-demouser
  name: k8scluster1-K8sHackTenant-demouser
current-context: k8scluster1-K8sHackTenant-demouser
kind: Config
preferences: {}
users:
- name: HPECP-demouser
  user:
    exec:
      apiVersion: client.authentication.k8s.io/v1beta1
      args:
      - hpecp
      - authenticate
      - gateway1.etc.fr.comm.hpecorp.net:8080
      - --hpecp-user=demouser
      - --hpecp-token=/api/v2/session/0c9ebaa8-0c00-4049-adcb-de43aba96e8b
      - --hpecp-token-expiry=1587074333
      - --force-reauth=false
      - --insecure-skip-tls-verify=true
      command: kubectl
      env: null


#### -3- Get the POD and Services for your deployed application:

In [10]:
kubectl get pod,service | grep ${username}

pod/hello-world-demouser-7d79c6c7dc-b2s2h   1/1     Running   0          116s
pod/hello-world-demouser-7d79c6c7dc-bcn8f   1/1     Running   0          116s
service/hello-world-service-demouser   NodePort   10.96.237.190   <none>        8080:30575/TCP   116s


In [11]:
kubectl describe service hello-world-service-${username}

Name:                     hello-world-service-demouser
Namespace:                k8shacktenant
Labels:                   hpecp.hpe.com/hpecp-internal-gateway=true
Annotations:              hpecp-internal-gateway/8080: gateway1.etc.fr.comm.hpecorp.net:10002
                          kubectl.kubernetes.io/last-applied-configuration:
                            {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{},"name":"hello-world-service-demouser","namespace":"k8shacktenant"},"spec"...
Selector:                 run=load-balancer-demouser
Type:                     NodePort
IP:                       10.96.237.190
Port:                     http-hello  8080/TCP
TargetPort:               8080/TCP
NodePort:                 http-hello  30575/TCP
Endpoints:                10.192.0.195:8080,10.192.1.72:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:
  Type    Reason  Age   From         Message
  ----    ------  ----  ----         -------
  Normal  HpeCp

#### -4- Get the service endpoints
HPE Container Platform automatically maps the NodePort Service endpoint to the HPE Container Platform gateway (proxy) host.
Access to application services running in containers is proxied via the gateway host and a port number greater than 10000.

In [12]:
myservice="hello-world-service-"${username}
appURL=$(kubectl describe service/"${myservice}" | grep gateway1 | awk '{print $3}')
myapp_endpoint="https://${appURL}"
echo "Your application service endpoint is: "$myapp_endpoint

Your application service endpoint is: https://gateway1.etc.fr.comm.hpecorp.net:10002


#### -5- Check your application is responding:

In [13]:
curl -k -s "${myapp_endpoint}"

Hello Kubernetes!

#### -6- Delete your stateless application deployment
Delete your application deployment and services using the REST API (POST) call below. The REST API call requires the **kubectl** operation type (delete) and the application manifest in a base64 encoded form.
After a minute or so, you should get the message: deployment deleted and service deleted.

In [14]:
curl -k -s --request POST "https://${controller_endpoint}/api/v2/k8scluster/${k8sClusterId}/kubectl" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"data": "'"$myapp"'",
"op": "delete"
}'

deployment.apps "hello-world-demouser" deleted
service "hello-world-service-demouser" deleted

## Deploying a KubeDirector Application

HPE has been working within the open source Kubernetes community to add capabilities that enable the running of **stateful** analytics workloads (data-intensive distributed applications) on Kubernetes.  The open source project is known as **Kubernetes Director** or **KubeDirector** for short.   
With KubeDirector, enterprises can deploy all their enterprise applications on a common Kubernetes framework.
KubeDirector is deployed as a custom controller (aka an operator) by default on every Kubernetes cluster managed by the HPE Container Platform.

For more information about KubeDirector, check out the HPE DEV portal [here](https://developer.hpe.com/platform/hpe-container-platform/home).

#### -1- List the installed KubeDirector Application
In the current release of HPE Container Platform, the Kubernetes cluster managed by HPE Controller platform has three (3) pre-configured KubeDirector Application types installed out of the box. A tenant user who would access the HPE Container Platform portal UI could see it:

<img src="Pictures/HPECP-KubeDirectorApp-GUI.png" height="800" width="600" align="left">

You can also get the list of KubeDirector Applications using kubectl command:

In [15]:
kubectl get kubedirectorapp

NAME                  AGE
centos7x              22d
ml-jupyter-notebook   22d
spark221e2            22d


A KubeDirector Application describes an application metadata: the service roles and endpoints, the Docker images, the app setup packages, the cardinality (minimum number of members in a role), and if appropriate, the root file system directories (e.g.: /etc, /bin, /opt, /var, /usr) of the containers to persist beyond the life span of the containers. This means stateful applications that require to write data to their root file systems can now successfully run on Kubernetes. Let's inspect a couple of KubeDirector Application types: 

In [17]:
kubectl describe kubedirectorapp ml-jupyter-notebook 

Name:         ml-jupyter-notebook
Namespace:    k8shacktenant
Labels:       <none>
Annotations:  token:
                execute this command to get the authentication token 'kubectl exec <pod-name> jupyter notebook list' and use this token in Jupyter noteboo...
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorApp
Metadata:
  Creation Timestamp:  2020-03-24T08:25:15Z
  Generation:          2
  Resource Version:    2874116
  Self Link:           /apis/kubedirector.hpe.com/v1beta1/namespaces/k8shacktenant/kubedirectorapps/ml-jupyter-notebook
  UID:                 cb7a6132-f321-45d6-81f5-635f0b79e61e
Spec:
  Config:
    Role Services:
      Role ID:  controller
      Service I Ds:
        jupyter-nb
    Selected Roles:
      controller
  Config Schema Version:   7
  Default Config Package:  <nil>
  Distro ID:               bluedata/tensorflow
  Label:
    Description:  TensorFlow GPU with Jupyter notebook
    Name:         TensorFlow + Jupyter
  Roles:
    Cardinality:

In [18]:
kubectl describe kubedirectorapp spark221e2

Name:         spark221e2
Namespace:    k8shacktenant
Labels:       <none>
Annotations:  <none>
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorApp
Metadata:
  Creation Timestamp:  2020-03-24T08:25:15Z
  Generation:          1
  Resource Version:    981021
  Self Link:           /apis/kubedirector.hpe.com/v1beta1/namespaces/k8shacktenant/kubedirectorapps/spark221e2
  UID:                 753618be-1edf-470c-bff5-385d0e76fafe
Spec:
  Config:
    Role Services:
      Role ID:  controller
      Service I Ds:
        ssh
        spark
        spark-master
        spark-worker
      Role ID:  worker
      Service I Ds:
        ssh
        spark-worker
      Role ID:  jupyter
      Service I Ds:
        ssh
        jupyter-nb
    Selected Roles:
      controller
      worker
      jupyter
  Config Schema Version:  7
  Distro ID:              bluedata/spark221e2
  Label:
    Description:  Spark 2.2.1 with Jupyter notebook
    Name:         Spark 2.2.1 + Jupyter
  Roles:
   

#### -2- Deploy Spark with Jupyter Notebook stateful application
A configuration manifest YAML file is used to create KubeDirector virtual clusters that instantiate the defined KubeDirector App type. The configuration file is used to describe the attributes of a given KubeDirector App.

We need to make sure the instance name of the KubeDirector Application is unique among your tenant. We also need to convert the application configuration manifest YAML file in a base64 encoded form.

In [19]:
sed -i "s/example/${username}/g" $sparkApp
cat $sparkApp

---
apiVersion: "kubedirector.hpe.com/v1beta1"
kind: "KubeDirectorCluster"
metadata: 
  name: "spark221e2-demouser"
  namespace: "k8shacktenant"
spec: 
  app: "spark221e2"
  appCatalog: "local"
  roles: 
    - 
      id: "controller"
      members: 1
      resources: 
        requests: 
          memory: "4Gi"
          cpu: "2"
        limits: 
          memory: "4Gi"
          cpu: "2"
      storage: 
        size: "10Gi"
    - 
      id: "worker"
      members: 2
      resources: 
        requests: 
          memory: "4Gi"
          cpu: "2"
        limits: 
          memory: "4Gi"
          cpu: "2"
      storage: 
        size: "10Gi"
    - 
      id: "jupyter"
      members: 1
      resources: 
        requests: 
          memory: "4Gi"
          cpu: "2"
        limits: 
          memory: "4Gi"
          cpu: "2"
      storage: 
        size: "10Gi"


The configuration manifest specifies the application instance name, the KubeDirectorApp type, the application service roles and their number of nodes and compute size, as well as the storage size as Spark requires persistent storage to work. 

*Note: In this deployment, the Spark KubeDirectorApp variant used here is a distributed implementation of Spark cluster where the master (Spark driver) and worker (Spark executor) services run on different cluster nodes (controller and workers). A separate Jupyter node (Id: jupyter) is used as interactive client to execute programs on Spark cluster.*

Let's deploy it now using the REST API call below. The REST API call requires the **kubectl** operation type (create or apply) and the application manifest in a base64 encoded form.

In [20]:
mysparkapp=$(base64 $sparkApp)
#echo $mysparkapp
curl -k -s --request POST "https://${controller_endpoint}/api/v2/k8scluster/${k8sClusterId}/kubectl" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"data": "'"$mysparkapp"'",
"op": "apply"
}'

kubedirectorcluster.kubedirector.hpe.com/spark221e2-demouser created

After a minute or so, you should get the message *kubedirectorcluster/Your-instance-name created*.  

Once you get this message, go to the next step.

#### -3- Inspect the deployed KubeDirector application virtual cluster 
Your application virtual cluster will be represented in the Kubernetes cluster by a resource of type **KubeDirectorCluster**, with the name that was indicated inside the YAML file used to create it. **A kubeDirectorCluster resource is an instantiation of a KubeDirector App**.

In [21]:
clusterName="spark221e2-${username}"
kubectl get kubedirectorcluster $clusterName

NAME                  AGE
spark221e2-demouser   6s


After creating the instance of the KubeDirector Application, you can use kubectl command below to observe its status and the standard Kubernetes resources that make up the application virtual cluster (statefulsets, pods, services, persistent volume claim if any) and any events logged against it.

The virtual cluster status indicates its overall "state" (top-level property of the status object). It should have a value of **"configured"**. The first time a virtual cluster of a given KubeDirector App type is created, it may take some minutes to reach "configured" state, as the relevant Docker image must be downloaded and imported.

In [27]:
kubectl describe kubedirectorcluster $clusterName

Name:         spark221e2-demouser
Namespace:    k8shacktenant
Labels:       <none>
Annotations:  kubectl.kubernetes.io/last-applied-configuration:
                {"apiVersion":"kubedirector.hpe.com/v1beta1","kind":"KubeDirectorCluster","metadata":{"annotations":{},"name":"spark221e2-demouser","names...
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorCluster
Metadata:
  Creation Timestamp:  2020-04-15T22:30:57Z
  Finalizers:
    kubedirector.hpe.com/cleanup
  Generation:        1
  Resource Version:  5655053
  Self Link:         /apis/kubedirector.hpe.com/v1beta1/namespaces/k8shacktenant/kubedirectorclusters/spark221e2-demouser
  UID:               f6060712-d478-4976-ad7f-400b06be8afd
Spec:
  App:          spark221e2
  App Catalog:  local
  Roles:
    Id:       controller
    Members:  1
    Resources:
      Limits:
        Cpu:     2
        Memory:  4Gi
      Requests:
        Cpu:     2
        Memory:  4Gi
    Storage:
      Size:                10Gi
      Stor

In [28]:
kubectl get all -l kubedirector.hpe.com/kdcluster=$clusterName

kubectl get pvc -l kubedirector.hpe.com/kdcluster=$clusterName

NAME               READY   STATUS    RESTARTS   AGE
pod/kdss-7dtqk-0   1/1     Running   0          31m
pod/kdss-7dtqk-1   1/1     Running   0          31m
pod/kdss-j2v6x-0   1/1     Running   0          31m
pod/kdss-mwkh4-0   1/1     Running   0          31m

NAME                     TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)                                                     AGE
service/kdhs-lbjmt       ClusterIP   None           <none>        8888/TCP                                                    31m
service/s-kdss-7dtqk-0   NodePort    10.96.141.31   <none>        22:32677/TCP,8081:31223/TCP                                 31m
service/s-kdss-7dtqk-1   NodePort    10.96.44.156   <none>        22:32143/TCP,8081:30019/TCP                                 31m
service/s-kdss-j2v6x-0   NodePort    10.96.215.65   <none>        22:30358/TCP,8080:31430/TCP,7077:31358/TCP,8081:31160/TCP   31m
service/s-kdss-mwkh4-0   NodePort    10.96.5.87     <none>        22:30390/TCP,8888:30227

Your KubeDirector application virtual cluster is made up of a **StatefulSet** per role (Spark controller, Spark workers, and jupyter), a **POD** (a cluster node) and a **NodePort Service** per service role member,  a **headless service** for the application cluster and a **Persistent Volume Claim (pvc)** per POD that requested persistent storage.  

* The ClusterIP service is the headless service required by a Kubernetes StatefulSet to work. It maintains a stable POD network identity (i.e.: persistence of the hostname of the PODs across PODs rescheduling).
* The NodePort service exposes an application service outside the Kubernetes cluster. 
HPE Container Platform automatically maps the NodePort Service endpoint to the HPE Container Platform gateway (proxy) host.
* HPE Container Platform defines HPE Data Fabric (aka MapR Data Platform) as **default** Kubernetes StorageClass for the Kubernetes Clusters managed by HPE Container Platform. HPE CP uses MapR Container Storage Interface (CSI) storage plugin to expose the HPE Data Fabric as storage provider to the Kubernetes containerized workloads (i.e.: PODs) that request persistent storage.

#### -4- Get the application service endpoints
To get a report on all services related to a specific virtual cluster, you can use a form of "kubectl describe" that matches against a value of the **kubedirector.hpe.com/kdcluster=YourClusterApplicationName** label.

In [29]:
#Jupyter Notebook service
sparkappJupyterURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway1 | grep 8888 | awk '{print $2}')
mysparkapp_Jupyter_endpoint="https://${sparkappJupyterURL}"

#Spark cluster Master service Web UI:
sparkappSparkURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway1 | grep 8080 | awk '{print $2}')
mysparkapp_Spark_endpoint="https://${sparkappSparkURL}"

#Spark cluster Worker service Web UI:
sparkappSparkWorkerURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway1 | grep 8081 | awk '{print $2}')
mysparkapp_SparkWorker_endpoint="https://${sparkappSparkWorkerURL}"

echo "Your application virtual cluster service endpoint (Jupyter) is: "$mysparkapp_Jupyter_endpoint
echo "Your application virtual cluster service endpoint (Spark Master) is: "$mysparkapp_Spark_endpoint
echo "Your application virtual cluster service endpoint (Spark Worker) is: "$mysparkapp_SparkWorker_endpoint

#kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName}

Your application virtual cluster service endpoint (Jupyter) is: https://gateway1.etc.fr.comm.hpecorp.net:10009
Your application virtual cluster service endpoint (Spark Master) is: https://gateway1.etc.fr.comm.hpecorp.net:10006
Your application virtual cluster service endpoint (Spark Worker) is: https://gateway1.etc.fr.comm.hpecorp.net:10007 gateway1.etc.fr.comm.hpecorp.net:10008 gateway1.etc.fr.comm.hpecorp.net:10011


Access to application services running in containers is proxied via the gateway host and a port number greater than 10000.

#### -5- Check the Spark cluster is running and Jupyter application service is responding

In [30]:
echo "Spark Master Web UI service response:"
curl -k -L -s -i "${mysparkapp_Spark_endpoint}" | grep "HTTP/1.1"

Spark Master Web UI service response:
HTTP/1.1 200 OK


In [ ]:
echo "Spark Worker Web UI service response:"
curl -k -L -s -i "${mysparkapp_SparkWorker_endpoint}" | grep "HTTP/1.1"

In [31]:
echo "Jupyter service response:"
curl -k -L -s -i "${mysparkapp_Jupyter_endpoint}" | grep "HTTP/1.1"

Jupyter service response:
HTTP/1.1 302 Found
HTTP/1.1 302 Found
HTTP/1.1 200 OK


#### -6- Connect to your application

You can connect to your Spark framework web UI dashboard and Jupyter Notebook and start your real time data processing.

Open a new tab in your browser, and connect to the service endpoint: https://77.158.163.130:YourPortNumber
where 77.158.163.130 is the NAT IP address of the HPE CP proxy gateway, port number is the port you get for your service endpoint. 

**Note:** In the first release of HPE Container Platform, the deployed instances of KubeDirectorApp containers do not have any login-capable accounts. Ask your administrator for Jupyter Notebook password.


**Spark Master Web UI:**

![Spark Cluster Master Web UI](Pictures/Spark-Cluster-Master-Web-UI.png)

**Spark Worker Web UI:** 

![Spark Cluster Worker Web UI](Pictures/Spark-Cluster-Worker-Web-UI.png)

**Jupyter Notebook:** 

![Spark client - Jupyter Notebook](Pictures/Spark-Jupyter-Notebook.png)

#### -7- Resizing the KubeDirector App cluster instance
To increase or decrease the number of members in a role, you would just have to edit the configuration YAML file and use the kubectl apply REST API call to apply the changes. The KubeDirector operator will manage the application cluster expansion or shrinkage for you. 

#### -8- Go through some cleanup

* First delete the kubedirector application cluster instance using the REST API (POST) call below. The REST API call requires the **kubectl** operation type (delete) and the application manifest in a base64 encoded form.
Deleting the KubeDirectorCluster resource will automatically delete all resources (the pods, services, pvc, statefulset) that compose the KubeDirector application virtual cluster.

In [32]:
curl -k -s --request POST "https://${controller_endpoint}/api/v2/k8scluster/${k8sClusterId}/kubectl" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"data": "'"$mysparkapp"'",
"op": "delete"
}'

kubedirectorcluster.kubedirector.hpe.com "spark221e2-demouser" deleted

* Although session have a time to live (TTL) of 24 hours, it is best practice in REST API programming to cleanup and delete those sessions when done. We can use a DELETE /api/v2/session/SessionId to achieve this.

In [33]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

HTTP/1.1 204 No Content
Access-Control-Allow-Origin: *
Content-Length: 0
Content-Type: text/plain
Date: Wed, 15 Apr 2020 23:11:14 GMT
Server: BlueData EPIC 5.0



The status *204 No Content* means the session object has been deleted.

* Finally, reset your applications YAML files

In [34]:
#reset the application deployment name in the YAML file
sed -i "s/${username}/example/g" $helloWorldApp
sed -i "s/${username}/example/g" $sparkApp
cat $helloWorldApp
cat $sparkApp

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world-example
spec:
  selector:
    matchLabels:
      run: load-balancer-example
  replicas: 2
  template:
    metadata:
      labels:
        run: load-balancer-example
    spec:
      containers:
        - name: hello-world-example
          image: gcr.io/google-samples/node-hello:1.0
          ports:
            - containerPort: 8080
              protocol: TCP
---
apiVersion: v1
kind: Service
metadata:
  name: hello-world-service-example
spec:
  selector:
    run: load-balancer-example
  ports:
  - name: http-hello
    protocol: TCP
    port: 8080
    targetPort: 8080
  type: NodePort
---
apiVersion: "kubedirector.hpe.com/v1beta1"
kind: "KubeDirectorCluster"
metadata: 
  name: "spark221e2-example"
  namespace: "k8shacktenant"
spec: 
  app: "spark221e2"
  appCatalog: "local"
  roles: 
    - 
      id: "controller"
      members: 1
      resources: 
        requests: 
          memory: "4Gi"
          cpu: "2"
        limi

## Summary

In this tutorial, you learned how **Kubernetes tenant** users can deploy, using REST API, both cloud-native stateless, microservices applications, and non-cloud-native distributed stateful AI/analytics KubeDirector applications on Kubernetes clusters managed by HPE Container Platform. You also used the standard Kubernetes command line (kubectl) as well as the HPE CP plugin to establish HPECP-authenticated kubectl requests in the context of your tenant user account. 